In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/chinese task/CLUENER2020')

In [3]:
#安装
!pip install transformers datasets pytorch-crf seqeval

In [4]:
import os
import json
import logging
import numpy as np
import pandas as pd
import config

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [5]:
#加载处理完的npz数据集
#不加allow_pickle=True会报错Object arrays cannot be loaded when allow_pickle=False，numpy新版本中默认为False。
train_data=np.load('./data/train.npz',allow_pickle=True)
val_data=np.load('./data/dev.npz',allow_pickle=True)
test_data=np.load('./data/test.npz',allow_pickle=True)

test_data.files

['words', 'labels']

In [6]:
#转换为dataframe格式
import pandas as pd
#补个随机frac
train_df=pd.concat([pd.DataFrame(train_data['words'],columns=['words']),
          pd.DataFrame(train_data['labels'],columns=['labels'])],axis=1).sample(frac=1.0)
#测试集和验证集不需要shuffle
val_df=pd.concat([pd.DataFrame(val_data['words'],columns=['words']),
          pd.DataFrame(val_data['labels'],columns=['labels'])],axis=1)#后面要进行预测对比标签，不宜shuffle

test_df=pd.concat([pd.DataFrame(test_data['words'],columns=['words']),
          pd.DataFrame(test_data['labels'],columns=['labels'])],axis=1)
#小样本测试
#train_df=train_df.iloc[:1000]
#val_df=val_df.iloc[:500]


#将训练验证集的BIOS标签转换为数字索引，此时word和labels已经对齐了
def trans(labels):
  labels=list(labels)
  nums=[]
  for label in labels:
    nums.append(config.label2id[label])
  return nums
    
train_df['labels']=train_df['labels'].map(lambda x: trans(x))
val_df['labels']=val_df['labels'].map(lambda x: trans(x))

test_df['labels']=test_df['labels'].map(lambda x: trans(x))
val_df

,words,labels
0,"[彭, 小, 军, 认, 为, ，, 国, 内, 银, 行, 现, 在, 走, 的, 是, ...","[7, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
1,"[温, 格, 的, 球, 队, 终, 于, 又, 踢, 了, 一, 场, 经, 典, 的, ...","[7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,"[突, 袭, 黑, 暗, 雅, 典, 娜, 》, 中, R, i, d, d, i, c, ...","[4, 14, 14, 14, 14, 14, 14, 14, 0, 7, 17, 17, ..."
3,"[郑, 阿, 姨, 就, 赶, 到, 文, 汇, 路, 排, 队, 拿, 钱, ，, 希, ...","[0, 0, 0, 0, 0, 0, 1, 11, 11, 0, 0, 0, 0, 0, 0..."
4,"[我, 想, 站, 在, 雪, 山, 脚, 下, 你, 会, 被, 那, 巍, 峨, 的, ...","[0, 0, 0, 0, 10, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
...,...,...
1338,"[在, 这, 个, 非, 常, 喜, 庆, 的, 日, 子, 里, ，, 我, 们, 首, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1339,"[姜, 哲, 中, ：, 公, 共, 之, 敌, 1, -, 1, 》, 、, 《, 神, ...","[6, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16..."
1340,"[目, 前, ，, 日, 本, 松, 山, 海, 上, 保, 安, 部, 正, 在, 就, ...","[0, 0, 0, 5, 15, 15, 15, 15, 15, 15, 15, 15, 0..."
1341,"[也, 就, 是, 说, 英, 国, 人, 在, 世, 博, 会, 上, 的, 英, 国, ...","[0, 0, 0, 0, 0, 0, 0, 0, 10, 20, 20, 0, 0, 0, ..."


## pandas数据装入datasets进行解码，之后方便直接pad labels。
## labels在后面装入dataloader的时候，不处理的话长度不一致，处理的话整理函数太麻烦。

In [7]:
from datasets import Dataset
from transformers import AutoTokenizer
#这里一定要选AutoTokenizer，如果是BertTokenizer，会提示bertbase没有word_ids方法。结果没用到
trains_ds=Dataset.from_pandas(train_df)
val_ds=Dataset.from_pandas(val_df)
test_ds=Dataset.from_pandas(test_df)

tokenizer=AutoTokenizer.from_pretrained(config.roberta_model,do_lower_case=True)

#tokenized_inputs=tokenizer(trains_ds["words"],padding=True,truncation=True,is_split_into_words=True)为啥这种是错的
tokenized_trains_ds=trains_ds.map(lambda examples:tokenizer(examples['words'],is_split_into_words=True,truncation=True,padding=True),batched=True)
tokenized_val_ds=val_ds.map(lambda examples:tokenizer(examples['words'],is_split_into_words=True,truncation=True,padding=True),batched=True)
tokenized_test_ds=test_ds.map(lambda examples:tokenizer(examples['words'],is_split_into_words=True,truncation=True,padding=True),batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [8]:
#在编码之后的datasets里面操作，得到的结果无法写入datasets，所以只好写到pandas文件里。
#将labels填充到和input_ids一样长（最长句子52，所以其实全部都填充到52）

def padding(data,pad):
  pad_labels=[]
  for ds in data:
    labels=ds['labels'] 
    mask=ds['attention_mask']
    label_ids=[pad]

    pad_length=len(mask)
    label_length=len(labels)
    
    label_ids=label_ids+labels+[pad]*(pad_length-label_length-1)
    pad_labels.append(label_ids)
  return pad_labels
#tokenized_trains_ds["pad_labels"]=pad_labels# Column 2 named labels expected length 10748 but got length 1000
"""
train_df['mask_labels']=padding(tokenized_trains_ds,-100)
val_df['mask_labels']=padding(tokenized_val_ds,-100)
test_df['mask_labels']=padding(tokenized_test_ds,-100)"""

train_df['pad_labels']=padding(tokenized_trains_ds,-1)
val_df['pad_labels']=padding(tokenized_val_ds,-1)
test_df['pad_labels']=padding(tokenized_test_ds,-1)
val_df

,words,labels,pad_labels
0,"[彭, 小, 军, 认, 为, ，, 国, 内, 银, 行, 现, 在, 走, 的, 是, ...","[7, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[-1, 7, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[温, 格, 的, 球, 队, 终, 于, 又, 踢, 了, 一, 场, 经, 典, 的, ...","[7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[-1, 7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
2,"[突, 袭, 黑, 暗, 雅, 典, 娜, 》, 中, R, i, d, d, i, c, ...","[4, 14, 14, 14, 14, 14, 14, 14, 0, 7, 17, 17, ...","[-1, 4, 14, 14, 14, 14, 14, 14, 14, 0, 7, 17, ..."
3,"[郑, 阿, 姨, 就, 赶, 到, 文, 汇, 路, 排, 队, 拿, 钱, ，, 希, ...","[0, 0, 0, 0, 0, 0, 1, 11, 11, 0, 0, 0, 0, 0, 0...","[-1, 0, 0, 0, 0, 0, 0, 1, 11, 11, 0, 0, 0, 0, ..."
4,"[我, 想, 站, 在, 雪, 山, 脚, 下, 你, 会, 被, 那, 巍, 峨, 的, ...","[0, 0, 0, 0, 10, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[-1, 0, 0, 0, 0, 10, 20, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
1338,"[在, 这, 个, 非, 常, 喜, 庆, 的, 日, 子, 里, ，, 我, 们, 首, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1339,"[姜, 哲, 中, ：, 公, 共, 之, 敌, 1, -, 1, 》, 、, 《, 神, ...","[6, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16...","[-1, 6, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16..."
1340,"[目, 前, ，, 日, 本, 松, 山, 海, 上, 保, 安, 部, 正, 在, 就, ...","[0, 0, 0, 5, 15, 15, 15, 15, 15, 15, 15, 15, 0...","[-1, 0, 0, 0, 5, 15, 15, 15, 15, 15, 15, 15, 1..."
1341,"[也, 就, 是, 说, 英, 国, 人, 在, 世, 博, 会, 上, 的, 英, 国, ...","[0, 0, 0, 0, 0, 0, 0, 0, 10, 20, 20, 0, 0, 0, ...","[-1, 0, 0, 0, 0, 0, 0, 0, 0, 10, 20, 20, 0, 0,..."


In [9]:
batch_size=16

#划分训练验证集
from sklearn.model_selection import train_test_split
from datasets import Dataset
from torch.nn.utils.rnn import pad_sequence


train_data,train_label,val_data,val_label=train_df['words'].iloc[:],train_df['pad_labels'].iloc[:],val_df['words'].iloc[:],val_df['pad_labels'].iloc[:]

test_data,test_label=(test_df['words'].iloc[:],test_df['pad_labels'].iloc[:])

#stratify=train_df['label'].iloc[:]报错:The least populated class in y has only 1 member,which is too few.
#The minimum number of groups for any class cannot be less than 2.估计是样本太少，分层抽取不可行。

#数据预处理

tokenizer=AutoTokenizer.from_pretrained(config.roberta_model,do_lower_case=True)
train_encoding=tokenizer(list(train_data),is_split_into_words=True,truncation=True,padding=True,return_tensors='pt')#训练集中划分的训练集
val_encoding=tokenizer(list(val_data),is_split_into_words=True,truncation=True,padding=True,return_tensors='pt')#训练集中划分的验证集
test_encoding=tokenizer(list(test_data),is_split_into_words=True,truncation=True,padding=True,return_tensors='pt')#测试集

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [10]:
#加载到datalodar并预处理
#数据集读取

from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch
class XFeiDataset(Dataset):
  def __init__(self,encodings,pad_labels):
    self.encodings=encodings
    self.pad_labels=pad_labels
  
  # 读取单个样本
  def __getitem__(self,idx):
    item={key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['pad_labels']=torch.tensor((self.pad_labels[idx]))
    item['mask']=(item['pad_labels']!=-1)
    return item
  
  def __len__(self):
    return len(self.pad_labels)

#def collate_fn

train_dataset=XFeiDataset(train_encoding,list(train_label))
val_dataset=XFeiDataset(val_encoding,list(val_label))
test_dataset=XFeiDataset(test_encoding,list(test_label))


from torch.utils.data import Dataset,DataLoader,TensorDataset

train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=False)
test_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)#test数据不能shuffle啊，真坑死我了

In [11]:
#for i in val_loader:
  #print(i)#输出5元组，input三兄弟和pad_labels和mask矩阵

In [12]:
from transformers import BertModel
from torch.nn.utils.rnn import pad_sequence
#初始化bert模型
from transformers import BertConfig
import torch.nn as nn
from torch.nn import LSTM
from torch.nn import functional as F 
from torchcrf import CRF

num_labels=31
dropout=0.1
#取0.1时，epoch=1，precision 0.68|recall 0.72|f1 0.70|acc 0.93
#选0.2训练loss更大。epoch1时precision 0.50|recall 0.60|f1 0.54|acc 0.90

class Bert_LSTM(nn.Module):
  def __init__(self):
    super(Bert_LSTM,self).__init__()
    self.num_labels=num_labels
    self.dropout=nn.Dropout(dropout)
    self.bert=BertModel.from_pretrained(config.roberta_model)
    for param in self.bert.parameters():
      param.requires_grad=True
    self.classifier=nn.Linear(1024,self.num_labels)
    self.crf=CRF(num_labels,batch_first=True)
    from torch.nn import functional as F

    self.bilstm=nn.LSTM(
        input_size=1024, 
        hidden_size=512, 
        batch_first=True,
        num_layers=2,
        dropout=0.5,  
        bidirectional=True)

  def forward(self,batch_seqs,batch_seq_masks,batch_seq_segments,pad_labels,mask):

    output=self.bert(input_ids=batch_seqs,attention_mask=batch_seq_masks,token_type_ids=batch_seq_segments)
    #pooler_output=output.pooler_output
    last_hidden_state=output.last_hidden_state
    last_hidden_state=self.dropout(last_hidden_state)
    #只有这种写法不会报错，如果是sequence_output,pooler_output=self.bert(**kwags)这种，sequence_output会报错str没有xxx属性。
    #貌似是bert输出有很多，直接用output.last_hidden_state来调用结果（估计是版本问题，坑），关键是输出要打印出来
    

    lstm_output,(hn,cn)=self.bilstm(last_hidden_state)
    #output为输出序列的隐藏层，hn为最后一个时刻的隐藏层，cn为最后一个时刻的隐藏细胞
    lstm_output=self.dropout(lstm_output)

    # 得到判别值
    logits=self.classifier(lstm_output)
    logits,pad_labels,mask=logits[:,1:,:],pad_labels[:,1:],mask[:,1:]#首个值不能是false，否则crf报错
    loss=self.crf(logits,pad_labels,mask)*(-1)
    
    return logits,loss

In [13]:
#加载模型
model=Bert_LSTM()
#model.load_state_dict(torch.load("/content/drive/MyDrive/chinese task/CLUENER2020/model/bert_lstm_crf_model"))
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext-large were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Bert_LSTM(
  (dropout): Dropout(p=0.1, inplace=False)
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 1024, padding_idx=1)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): L

In [14]:
#定义优化器
epoch=10
lr=3e-5

from transformers import AdamW,get_scheduler

train_steps_per_epoch=len(train_loader)
num_training_steps=train_steps_per_epoch*epoch

#定义各模块参数
bert_parameters=list(model.bert.named_parameters())
lstm_parameters=list(model.bilstm.named_parameters())
classifier_parameters=list(model.classifier.named_parameters())
no_decay=['bias','LayerNorm.weight']

#bert模型、lstm模型、nn.linear的学习率分离，后两个是bert的3倍
optimizer_grouped_parameters=[
    {'params':[p for n,p in bert_parameters if not any(nd in n for nd in no_decay)],
      'lr':lr,'weight_decay':0.01},
    {'params':[p for n,p in bert_parameters if any(nd in n for nd in no_decay)],
      'lr':lr,'weight_decay':0.0},
    {'params':[p for n,p in lstm_parameters if not any(nd in n for nd in no_decay)],
      'lr':lr*3,'weight_decay':0.01},
    {'params':[p for n,p in lstm_parameters if any(nd in n for nd in no_decay)],
      'lr':lr*3,'weight_decay': 0.0},
    {'params':[p for n,p in classifier_parameters if not any(nd in n for nd in no_decay)],
      'lr':lr*3,'weight_decay':0.01},
    {'params':[p for n,p in classifier_parameters if any(nd in n for nd in no_decay)],
      'lr':lr*3,'weight_decay':0.0}]

optimizer=AdamW(optimizer_grouped_parameters,lr=lr,eps=1e-8)
#使用线性衰减学习率
lr_scheduler=get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps)

In [15]:
#将crf.decode预测值用0进行pad之后转为tensor
def pad_result(data,pad_labels):
  pad_pred=[]
  max_len=pad_labels.shape[1]
  for pred in data:
    pad_length=max_len-len(pred)

    label_ids=pred+[0]*(pad_length)
    pad_pred.append(label_ids)
    
  return pad_pred

#pred=torch.tensor(pred)必须在函数外，否则报错tensor没有append

#编写评价方法
from datasets import load_metric
metric=load_metric("seqeval")
import numpy as np

label_list= [label for label,id in list(config.label2id.items())]

def compute_metrics(y_pred,y_true):
  predictions,labels=y_pred,y_true

  # 掉特殊字符处的值，不作比较。
  true_predictions=[
    [label_list[p] for (p,l) in zip(prediction, label) if l !=-1]
    for prediction,label in zip(predictions, labels)
  ]
  true_labels=[
    [label_list[l] for (p,l) in zip(prediction, label) if l !=-1]
    for prediction,label in zip(predictions, labels)
  ]

  results = metric.compute(predictions=true_predictions,references=true_labels)
  return results


In [16]:
#编写训练和验证循环
import time
import numpy as np
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
from torch.nn import functional as F
from torchcrf import CRF
#加载进度条
from tqdm.auto import tqdm

num_training_steps=train_steps_per_epoch*epoch

progress_bar=tqdm(range(num_training_steps))

def train_and_eval(epoch):
  for i in range(epoch):
    """训练模型"""
    start=time.time()
    model.train()
    print("***** Running training epoch {} *****".format(i+1))
    train_loss_sum=0.0
    for idx,batch in enumerate(train_loader):
      input_ids=batch['input_ids'].to(device)
      attention_mask=batch['attention_mask'].to(device)
      token_type_ids=batch['token_type_ids'].to(device)
      pad_labels=batch['pad_labels'].to(device)
      mask=batch['mask'].to(device)


      #计算输出和loss
      logits,loss=model(input_ids,attention_mask,token_type_ids,pad_labels,mask)
      loss.backward()

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()  
      progress_bar.update(1)

      train_loss_sum+=loss.item()
      if (idx+1)%(len(train_loader)//5)==0: # 只打印五次结果
        print("Epoch {:03d} | Step {:04d}/{:04d} | Loss {:.4f} | Time {:.4f} | Learning rate = {} \n".format(
                  i+1,idx+1,len(train_loader),train_loss_sum/(idx+1),time.time()-start,optimizer.state_dict()['param_groups'][0]['lr']))
      
      #验证模型
    model.eval()
    y_pred,y_true=[],[]
    best_f1,total_eval_loss=0,0
    total_eval_accuracy,total,acc=0,0,0
    
    for batch in val_loader:
      with torch.no_grad():#只有这一块是不需要求导的
      
        input_ids=batch['input_ids'].to(device)
        attention_mask=batch['attention_mask'].to(device)
        token_type_ids=batch['token_type_ids'].to(device)
        pad_labels=batch['pad_labels'].to(device)
        mask=batch['mask'].to(device)
        logits,loss=model(input_ids,attention_mask,token_type_ids,pad_labels,mask)#都是去掉首个-100的结果                      
           
      total_eval_loss+=loss.item()
      

      pad_labels,mask=pad_labels[:,1:].to(device),mask[:,1:].to(device)
      pred=model.crf.decode(logits,mask)
      pred=torch.tensor(pad_result(pred,pad_labels)).to(device)#预测值经过pad_pred函数pad成batch_size*max_length-1，再转为tensor
      acc+=(pred[mask]==pad_labels[mask]).sum().item()#只计算没有mask的单词的准确率,mask在外面似乎accs0.93不准。
      total+=mask.sum().item()
      total_eval_accuracy=acc/total

      y_pred.extend(pred.cpu().numpy().tolist())#将每个把batch结果依次加入总列表
      y_true.extend(pad_labels.cpu().numpy().tolist())
    results=compute_metrics(y_pred,y_true)
    f1=results["overall_f1"]    
    
    if f1>best_f1:
      best_f1=f1
      torch.save(model.state_dict(),"./bert_lstm_crf/blf_model")
    
    print("precision {:.2f}|recall {:.2f}|f1 {:.4f}|acc {:.2f}".format(results["overall_precision"],results["overall_recall"],results["overall_f1"],results["overall_accuracy"]))
    print("Average val loss:%.2f"%(total_eval_loss),"sklearn_acc:%.2f"%(total_eval_accuracy))
    print("time costed={}s \n".format(round(time.time()-start,5)))
    print("-------------------------------")

  0%|          | 0/6720 [00:00<?, ?it/s]

## 对比bert的token分类任务头

In [17]:
train_and_eval(epoch)
"""
预测值不包括实际值会报错
UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))"""

***** Running training epoch 1 *****


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


Epoch 001 | Step 0134/0672 | Loss 850.4533 | Time 160.0524 | Learning rate = 2.9401785714285713e-05 

Epoch 001 | Step 0268/0672 | Loss 755.6008 | Time 321.1079 | Learning rate = 2.880357142857143e-05 

Epoch 001 | Step 0402/0672 | Loss 693.5408 | Time 482.2713 | Learning rate = 2.8205357142857143e-05 

Epoch 001 | Step 0536/0672 | Loss 626.1115 | Time 643.2802 | Learning rate = 2.7607142857142855e-05 

Epoch 001 | Step 0670/0672 | Loss 554.6280 | Time 804.1686 | Learning rate = 2.7008928571428574e-05 

precision 0.55|recall 0.65|f1 0.59|acc 0.90
Average val loss:16898.68 sklearn_acc:0.90
time costed=847.54825s 

-------------------------------
***** Running training epoch 2 *****
Epoch 002 | Step 0134/0672 | Loss 189.6032 | Time 161.3824 | Learning rate = 2.6401785714285714e-05 

Epoch 002 | Step 0268/0672 | Loss 174.3528 | Time 321.8987 | Learning rate = 2.580357142857143e-05 

Epoch 002 | Step 0402/0672 | Loss 166.6937 | Time 482.3897 | Learning rate = 2.5205357142857145e-05 

Epoch

'\n预测值不包括实际值会报错\nUndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.\n  _warn_prf(average, modifier, msg_start, len(result))'

In [ ]:
#torch.save(model.state_dict(),"./bert_lstm_crf/finall_blf_model")
model.load_state_dict(torch.load("./bert_lstm_crf/blf_model"))
model.to(device)

In [27]:
#编写predict函数
def predict(model,data_loader):#参数名为data时加载训练好的模型来预测报错，原模型不报错
  model.eval()
  y_pred,y_true,predictions= [],[],[]#pad后的预测结果、真实标签和没有pad的标签list
  for batch in data_loader:
    with torch.no_grad():#只有这一块是不需要求导的
      
      input_ids=batch['input_ids'].to(device)
      attention_mask=batch['attention_mask'].to(device)
      token_type_ids=batch['token_type_ids'].to(device)
      pad_labels=batch['pad_labels'].to(device)#用1填充的标签，用来计算logits和loss
      mask=batch['mask'].to(device)#mask矩阵，用来计算logits和loss、crf解码结果
      logits,loss=model(input_ids,attention_mask,token_type_ids,pad_labels,mask)                      
    
    pad_labels,mask=pad_labels[:,1:].to(device),mask[:,1:].to(device)
    prediction=model.crf.decode(logits,mask)#解码后得出真实tokens的预测结果list

    #预测值只是列表，pad之后加入总的预测列表，用于评价compute_metrics
    pad_pred=torch.tensor(pad_result(prediction,pad_labels)).to(device)
    y_pred.extend(pad_pred.cpu().numpy().tolist())
    y_true.extend(pad_labels.cpu().numpy().tolist())#pad_labels为真实标签

    predictions.extend(prediction)#这个是不用pad的预测结果，用于最终提交结果
      
  return y_pred,y_true,predictions

In [28]:
#预测验证集结果，查看各个tokens类别的指标
y_pred,y_true,predictions=predict(model,val_loader)
results=compute_metrics(y_pred,y_true)
#将结果排序查看
result_df=pd.DataFrame(results)
result_df.stack().unstack(0).sort_values(by=['f1'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


,precision,recall,f1,number
address,0.555288,0.619303,0.585551,373.000000
scene,0.697115,0.693780,0.695444,209.000000
overall_precision,0.747256,0.747256,0.747256,0.747256
organization,0.721805,0.784741,0.751958,367.000000
overall_f1,0.771725,0.771725,0.771725,0.771725
government,0.739130,0.825911,0.780115,247.000000
position,0.774554,0.801386,0.787741,433.000000
book,0.775000,0.805195,0.789809,154.000000
company,0.759124,0.825397,0.790875,378.000000
overall_recall,0.797852,0.797852,0.797852,0.797852


In [31]:
#将验证集真实标签和预测结果进行对比展示
val_df['preds']=pd.Series(predictions)
val_df.to_csv('./bert_lstm_crf/val_1221.csv')
val_df=val_df.drop(["pad_labels"],axis=1)
val_df

,words,labels,preds
0,"[彭, 小, 军, 认, 为, ，, 国, 内, 银, 行, 现, 在, 走, 的, 是, ...","[7, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[7, 17, 17, 0, 0, 0, 3, 13, 13, 13, 0, 0, 0, 0..."
1,"[温, 格, 的, 球, 队, 终, 于, 又, 踢, 了, 一, 场, 经, 典, 的, ...","[7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,"[突, 袭, 黑, 暗, 雅, 典, 娜, 》, 中, R, i, d, d, i, c, ...","[4, 14, 14, 14, 14, 14, 14, 14, 0, 7, 17, 17, ...","[6, 16, 16, 16, 16, 16, 16, 16, 0, 7, 17, 17, ..."
3,"[郑, 阿, 姨, 就, 赶, 到, 文, 汇, 路, 排, 队, 拿, 钱, ，, 希, ...","[0, 0, 0, 0, 0, 0, 1, 11, 11, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 1, 11, 11, 0, 0, 0, 0, 0, 0..."
4,"[我, 想, 站, 在, 雪, 山, 脚, 下, 你, 会, 被, 那, 巍, 峨, 的, ...","[0, 0, 0, 0, 10, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
1338,"[在, 这, 个, 非, 常, 喜, 庆, 的, 日, 子, 里, ，, 我, 们, 首, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1339,"[姜, 哲, 中, ：, 公, 共, 之, 敌, 1, -, 1, 》, 、, 《, 神, ...","[6, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16...","[6, 17, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16..."
1340,"[目, 前, ，, 日, 本, 松, 山, 海, 上, 保, 安, 部, 正, 在, 就, ...","[0, 0, 0, 5, 15, 15, 15, 15, 15, 15, 15, 15, 0...","[0, 0, 0, 5, 15, 15, 15, 15, 15, 15, 15, 15, 0..."
1341,"[也, 就, 是, 说, 英, 国, 人, 在, 世, 博, 会, 上, 的, 英, 国, ...","[0, 0, 0, 0, 0, 0, 0, 0, 10, 20, 20, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 20..."


## 用模型预测验证集结果，与原标签对比

In [32]:
y_pred,y_true,predictions=predict(model,test_loader)
pd.DataFrame({'label':predictions}).to_csv('./bert_lstm_crf/submit1222.csv',index=None)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
